If you have a gpu access with colab you can get things accomplished faster. If you do not have one forget it, we will assume that all of us a free version and not the pro one.

In [1]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
print(tf.__version__)

#import tensorflowjs as tfjs

import numpy
import tensorflow.keras as keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

2.9.2


In [3]:
# Training Generator
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/orange-diseases-dataset/dataset/train',
                                                target_size=(224,224),
                                                color_mode='rgb',
                                                batch_size=32,
                                                class_mode='categorical',
                                                shuffle=True)

Found 1164 images belonging to 4 classes.


In [4]:
num_classes = 4
prediction_dict = {0: 'blackspot', 1: 'canker', 2: 'fresh', 3: 'grenning'}

Here we will be using transfer learning approach and hence the extra layers

In [5]:
base_model=keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224,224,3),weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

# Add Extra Layers to Model
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(num_classes,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)

9406464/9406464 [==============================] - 0s 0us/step


In [6]:
# Check layers no. & name
for i,layer in enumerate(model.layers):
    print(i,layer.name)

0 input_1
1 Conv1
2 bn_Conv1
3 Conv1_relu
4 expanded_conv_depthwise
5 expanded_conv_depthwise_BN
6 expanded_conv_depthwise_relu
7 expanded_conv_project
8 expanded_conv_project_BN
9 block_1_expand
10 block_1_expand_BN
11 block_1_expand_relu
12 block_1_pad
13 block_1_depthwise
14 block_1_depthwise_BN
15 block_1_depthwise_relu
16 block_1_project
17 block_1_project_BN
18 block_2_expand
19 block_2_expand_BN
20 block_2_expand_relu
21 block_2_depthwise
22 block_2_depthwise_BN
23 block_2_depthwise_relu
24 block_2_project
25 block_2_project_BN
26 block_2_add
27 block_3_expand
28 block_3_expand_BN
29 block_3_expand_relu
30 block_3_pad
31 block_3_depthwise
32 block_3_depthwise_BN
33 block_3_depthwise_relu
34 block_3_project
35 block_3_project_BN
36 block_4_expand
37 block_4_expand_BN
38 block_4_expand_relu
39 block_4_depthwise
40 block_4_depthwise_BN
41 block_4_depthwise_relu
42 block_4_project
43 block_4_project_BN
44 block_4_add
45 block_5_expand
46 block_5_expand_BN
47 block_5_expand_relu
48 b

In [7]:
# set extra layers to trainable (layer #155~159)
for layer in model.layers[:155]:
    layer.trainable=False
for layer in model.layers[155:]:
    layer.trainable=True

# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [8]:
# Train Model (target is loss <0.01)
num_epochs = 12
step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, epochs=num_epochs)


<ipython-input-8-c8ea5399a984>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, epochs=num_epochs)


Epoch 1/12
36/36 [==============================] - 336s 9s/step - loss: 0.2772 - accuracy: 0.8922
Epoch 2/12
36/36 [==============================] - 41s 1s/step - loss: 0.0801 - accuracy: 0.9682
Epoch 3/12
36/36 [==============================] - 43s 1s/step - loss: 0.0352 - accuracy: 0.9903
Epoch 4/12
36/36 [==============================] - 41s 1s/step - loss: 0.0283 - accuracy: 0.9920
Epoch 5/12
36/36 [==============================] - 41s 1s/step - loss: 0.0170 - accuracy: 0.9938
Epoch 6/12
36/36 [==============================] - 41s 1s/step - loss: 0.0144 - accuracy: 0.9938
Epoch 7/12
36/36 [==============================] - 41s 1s/step - loss: 0.0131 - accuracy: 0.9956
Epoch 8/12
36/36 [==============================] - 43s 1s/step - loss: 0.0257 - accuracy: 0.9920
Epoch 9/12
36/36 [==============================] - 41s 1s/step - loss: 0.0139 - accuracy: 0.9947
Epoch 10/12
36/36 [==============================] - 43s 1s/step - loss: 0.0051 - accuracy: 0.9982
Epoch 11/12
36/36 

This is a function to reshape the image as per the input size specs of the netwrok

In [9]:
def prepare_image224(filepath):
   img = image.load_img(filepath, target_size=(224, 224))
   img_array = image.img_to_array(img)
   img_array_expanded_dims = numpy.expand_dims(img_array, axis=0)
   return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [10]:
preprocessed_image = prepare_image224("/content/drive/MyDrive/orange-diseases-dataset/dataset/test/canker/cancro_teste (99).jpg")
predictions = model.predict(preprocessed_image)
maxindex = int(numpy.argmax(predictions))
print(predictions[0][maxindex],prediction_dict[maxindex])

1/1 [==============================] - 1s 913ms/step
0.5897055 canker


In [11]:
export_path_keras = "/content/drive/MyDrive/{}.h5".format("analytics_vidya_jan_2023")
print(export_path_keras)

model.save(export_path_keras)


/content/drive/MyDrive/analytics_vidya_jan_2023.h5


We saved the model, now lets load the model again to see it works,

In [13]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [14]:
export_path_keras = "/content/drive/MyDrive/{}.h5".format("analytics_vidya_jan_2023")

reloaded = tf.keras.models.load_model(
  export_path_keras, 
  # `custom_objects` tells keras how to load a `hub.KerasLayer`
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [15]:
preprocessed_image = prepare_image224("/content/drive/MyDrive/orange-diseases-dataset/dataset/test/canker/cancro_teste (99).jpg")
predictions = reloaded.predict(preprocessed_image)
maxindex = int(numpy.argmax(predictions))
print(predictions[0][maxindex],prediction_dict[maxindex])

1/1 [==============================] - 1s 837ms/step
0.5897055 canker


Lets save the model

In [ ]:
saved_model_dir = '/content/drive/MyDrive/lite-android-test'
tf.saved_model.save(reloaded, saved_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
with open(f'{saved_model_dir}/model02.tflite', 'wb') as f:
 f.write(tflite_model)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open(f'{saved_model_dir}/labels02.txt', 'w') as f:
 f.write(labels)

The best part, do not wait to deploy the model on an mobile app to see the results. Its far more easy.


In [16]:
!pip install tflite-runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.6 MB/s eta 0:00:00


In [24]:
import tflite_runtime.interpreter as tflite
from PIL import Image
import numpy as np
import time

In [20]:
def load_labels(filename):
  with open(filename, 'r') as f:
    return [line.strip() for line in f.readlines()]

In [23]:
interpreter = tflite.Interpreter(model_path="/content/drive/MyDrive/lite-android-test/model02.tflite")
input_details = interpreter.get_input_details()

input_shape = input_details[0]["shape"]
print(input_shape)
#batch size = input_shape[0], this can be ignored here. We will only be using 1 img.
height, width, channels = input_shape[1], input_shape[2], input_shape[3]

#if channels == 1 then greyscale
grayscale = channels == 1
print(channels)
#load the input image
image_path = "/content/drive/MyDrive/orange-diseases-dataset/dataset/test/canker/cancro_teste (99).jpg"
image = Image.open(image_path).resize((width, height))
print(image)
if not grayscale:
    #image = image.convert("LA")
    #extend dims so model can expect it.

    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # check the type of the input tensor
    print(input_details[0]['dtype'])
    floating_model = input_details[0]['dtype'] == np.float32

    input_data = np.expand_dims(image, axis=0)

    if floating_model:
        input_data = np.float32(input_data)

    interpreter.set_tensor(input_details[0]['index'], input_data)

    start_time = time.time()
    interpreter.invoke()
    stop_time = time.time()

    output_data = interpreter.get_tensor(output_details[0]['index'])

    # interpreter.set_tensor(input_details[0]['index'], image)
    # interpreter.invoke()
    # output_data = interpreter.get_tensor(output_details[0]['index'])

    # print(output_data)
    results = np.squeeze(output_data)

    top_k = results.argsort()[-5:][::-1]
    labels = load_labels("/content/drive/MyDrive/lite-android-test/labels02.txt")
    for i in top_k:
        if floating_model:
            print('{:0.2f}: {}'.format(float(results[i]), labels[i]))
        else:
            print('{:0.2f}: {}'.format(float(results[i] / 255.0), labels[i]))

    print('time: {:.3f}ms'.format((stop_time - start_time) * 1000))

[  1 224 224   3]
3
<PIL.Image.Image image mode=RGB size=224x224 at 0x7FE9C928D910>
<class 'numpy.float32'>
0.82: canker
0.18: fresh
0.00: grenning
0.00: blackspot
time: 22.771ms
